In [12]:
asdf

NameError: name 'asdf' is not defined

# Imports

In [1]:
import pathlib as pl
import nesdis_aws
import nesdis_gml_synergy.satlab as ngs
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
import os
os.getpid()

3723

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# warnings.filterwarnings("default")

In [4]:
plt.rcParams['figure.dpi'] = 200

# download some data

In [5]:
reload(nesdis_aws.nesdis_aws)
reload(nesdis_aws)

<module 'nesdis_aws' from '/mnt/telg/prog/nesdis_aws/nesdis_aws/__init__.py'>

In [ ]:
# product = 'ACHa' #FDC
path2folder_local=f'/mnt/telg/data/nesdis/goes/ABI_L2_{product}/'
pl.Path(path2folder_local).mkdir()
query = nesdis_aws.AwsQuery(path2folder_local=path2folder_local,
                            satellite='16',
                            product=f'ABI-L2-{product}',
                            scan_sector='C',
                            # start='2020-09-30 18:00:00',
                            # end='2020-09-30 19:35:00',
                            start='2018-08-01 18:00:00',
                            # end='2018-11-01 00:00:00',
                            end='2018-08-01 18:05:00',
                            verbose = True
                           )


In [ ]:
query.workplan

In [ ]:
print(query.info_on_current_query())

In [ ]:
query.product_available_since

In [ ]:
query.download()


# open

In [ ]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
stations = surfrad.network.stations

In [ ]:
reload(ngs)

In [ ]:
# sat_ds = ngs.open_file('/mnt/telg/data/nesdis/goes/ABI_L2_ADP/OR_ABI-L2-ADPC-M6_G16_s20211431801169_e20211431803542_c20211431804531.nc', verbose=True)

In [ ]:
sat_ds.valid_2D_variables

In [ ]:
list(sat_ds.ds.variables)

In [ ]:
proj = sat_ds.project_on_sites(stations)

In [ ]:
proj.projection2area.sel(site = 'BND', stats = 'mean', radius = 50, 
                         # data_quality = 'good'
                        )

In [ ]:
proj.projection2point

# Plot on map

In [ ]:
scale = 0.5
width=2800000 *scale
height=1700000 * scale
a, bmap = surfrad.network.stations.Table_Mountain.plot(width=width, height=height)
bmap, pc, cb = sat_ds.plot('Temp', bmap = bmap)
bmap.drawcounties()
# f = plt.gcf()
# pc.set_clim(vmax= 10)

# Download and project

In [5]:
import pathlib as pl
import nesdis_aws
import nesdis_gml_synergy.satlab as ngs
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
import os
os.getpid()

3723

In [6]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad

stations = surfrad.network.stations

In [13]:
def function(row, stations):#, alt_var = None):
    # if isinstance(alt_var, type(None)):
    #     alt_var = variable
    # read the file
    ngsinst = ngs.open_file(row.path2file_local)
    
    # project to stations
    projection = ngsinst.project_on_sites(stations)

    # merge closest gridpoint and area
    point = projection.projection2point.copy()#.sel(site = 'TBL')
    point['DQF'] = point.DQF.astype(int) # for some reason this was float64... because there are some nans in there
    
    # change var names to distinguish from area
    for var in ngsinst.valid_2D_variables:
        point = point.rename({var: f'{var}_on_pixel',})
    point = point.rename({'DQF': 'DQF_on_pixel'})
    
    # merge aerea and point
    ds = projection.projection2area.merge(point)#.rename({alt_var: f'{alt_var}_on_pixel', 'DQF': 'DQF_on_pixel'}))
    
    # add a time stamp
    dt = pd.Series([pd.to_datetime(ngsinst.ds.attrs['time_coverage_start']), pd.to_datetime(ngsinst.ds.attrs['time_coverage_end'])]).mean().to_datetime64()
    ds = ds.expand_dims({'datetime': [dt]}, )
    
    # there was another time coordinate without dimention though ... dropit
    ds = ds.drop_vars('t')

    # global attribute
    ds.attrs['info'] = ('This file contains a projection of satellite data onto SURFRAD sites.\n'
                         'It includes the closest pixel data as well as the average over circular\n'
                         'areas with various radii. Note, for the averaged data only data is\n'
                         'considered with a qulity flag given by the prooduct class in the\n'
                         'nesdis_gml_synergy package.')
    

    # save2file
    ds.to_netcdf(row.path2file_local_processed)
    return 

In [14]:
product = 'DSR'
# alt_var = 'BCM'# None, if the varialbe name is not equal to product name
query = nesdis_aws.nesdis_aws.AwsQuery(
                                        path2folder_local=f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}/',
                                        satellite='16',
                                        product=f'ABI-L2-{product}',
                                        scan_sector='C',
                                        # start='2020-03-01 18:00:00',
                                        # end='2020-03-01 18:05:00',
                                        start='2020-03-01 00:00:00',
                                        end='2020-06-01 00:00:00',
                                        # start='2018-08-01 00:00:00',
                                        # end='2018-11-01 00:00:00',
                                        process = dict(#concatenate = 'daily',
                                                       function = lambda row: function(row, stations), #,, alt_var),
                                                       prefix = f'ABI_L2_{product}_projected2surfrad',
                                                       path2processed = f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}_projected2surfrad/'),
                                        # keep_files=True
                                      )

# pl.Path(f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}').mkdir()
# pl.Path(f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}_projected2surfrad/').mkdir()

In [15]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed


In [16]:
for i in query.workplan.iloc[0]:
    print(i)

IndexError: single positional indexer is out-of-bounds

In [ ]:
query.process(raise_exception=True)

In [9]:
ds = xr.open_dataset('/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_DSR_projected2surfrad/ABI_L2_DSR_projected2surfrad_20200301_180114.nc')

In [17]:
ds.DSR.sel(site = 'TBL', stats = 'mean', radius = 10)

<xarray.DataArray 'DSR' (datetime: 1, data_quality: 1)>
array([[411.52142]], dtype=float32)
Coordinates:
  * datetime      (datetime) datetime64[ns] 2020-03-01T18:02:32.849999872
    radius        int64 10
    stats         <U4 'mean'
    site          <U3 'TBL'
  * data_quality  (data_quality) object 'high'
Attributes:
    long_name:            ABI L2+ Downward Shortwave Radiation: Surface
    standard_name:        surface_downwelling_shortwave_flux_in_air
    valid_range:          [ 0 -6]
    units:                W m-2
    resolution:           lon: 0.25 degree lat: 0.25 degree
    grid_mapping:         goes_lat_lon_projection
    cell_methods:         retrieval_local_zenith_angle: point (good or degrad...
    ancillary_variables:  DQF

In [16]:
ds.DSR_on_pixel.sel(site = 'TBL')

<xarray.DataArray 'DSR_on_pixel' (datetime: 1)>
array([411.52142], dtype=float32)
Coordinates:
  * datetime  (datetime) datetime64[ns] 2020-03-01T18:02:32.849999872
    site      <U3 'TBL'
Attributes:
    long_name:            ABI L2+ Downward Shortwave Radiation: Surface
    standard_name:        surface_downwelling_shortwave_flux_in_air
    valid_range:          [ 0 -6]
    units:                W m-2
    resolution:           lon: 0.25 degree lat: 0.25 degree
    grid_mapping:         goes_lat_lon_projection
    cell_methods:         retrieval_local_zenith_angle: point (good or degrad...
    ancillary_variables:  DQF
    coordinates:          retrieval_local_zenith_angle quantitative_local_zen...

# Concatenate

In [19]:
reload(ngs)

<module 'nesdis_gml_synergy.satlab' from '/mnt/telg/prog/nesdis_gml_synergy/nesdis_gml_synergy/satlab.py'>

In [23]:
cc = ngs.Concatonator(path2scraped_files='/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_DSR_projected2surfrad/',
                 path2concat_files ='/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_DSR_projected2surfrad_concat/',
                 datetime_format='ABI_L2_DSR_projected2surfrad_%Y%m%d_%H%M%S.nc',
                 test = False,)

In [24]:
cc.workplan

,path2scraped_files,datetime,date,product_name,path2concat_files,p2rf_exists


In [22]:
ds=cc.concat_and_save(save=True, test=False, verbose=False)